<a href="https://colab.research.google.com/github/elichen/aoc2024/blob/main/Day_9_Disk_Fragmenter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
input = "2333133121414131402"

In [46]:
input = open("input.txt").read().rstrip()

In [48]:
def fill_empty_sections(lengths):
    # Convert from string if needed
    if isinstance(lengths, str):
        lengths = list(map(int, lengths))

    # Build initial list of (id, length), alternating between non-empty and empty
    result = []
    next_id = 0
    is_nonempty = True
    for length in lengths:
        section_id = next_id if is_nonempty else -1
        result.append((section_id, length))
        if is_nonempty:
            next_id += 1
        is_nonempty = not is_nonempty

    i = 0
    while i < len(result):
        if result[i][0] == -1:
            empty_len = result[i][1]
            if empty_len == 0:
                # Zero-length empty: remove it and continue
                del result[i]
                continue

            # Check if we can fully fill this empty section from the trailing sections
            # Calculate total available length from trailing non-empty sections
            total_available = 0
            for sec_id, sec_len in reversed(result[i+1:]):
                if sec_id == -1:
                    # Ignore trailing empty sections
                    continue
                total_available += sec_len
                if total_available >= empty_len:
                    break

            if total_available < empty_len:
                # Not enough to fill - just remove the empty section
                del result[i]
                # Don't advance i since we removed the section at i
            else:
                # We can fill it. Let's do the fill now.
                to_fill = empty_len
                fill_pieces = []

                # We'll consume from the end towards the front
                pos = len(result) - 1
                while to_fill > 0 and pos > i:
                    sec_id, sec_len = result[pos]
                    if sec_id == -1:
                        # Discard trailing empty sections
                        del result[pos]
                        pos -= 1
                        continue
                    if sec_len <= to_fill:
                        # Use all of this section
                        fill_pieces.append((sec_id, sec_len))
                        to_fill -= sec_len
                        del result[pos]
                        pos -= 1
                    else:
                        # Use part of this section
                        fill_pieces.append((sec_id, to_fill))
                        remainder = sec_len - to_fill
                        result[pos] = (sec_id, remainder)
                        to_fill = 0

                # IMPORTANT: Do NOT reverse fill_pieces here.
                # Insert them in the order they were taken.
                result[i:i+1] = fill_pieces
                i += len(fill_pieces)

        else:
            i += 1

    # Remove any trailing empty sections if any remain
    while result and result[-1][0] == -1:
        result.pop()

    return result

print(fill_empty_sections("12345"))
print(fill_empty_sections("2333133121414131402"))

[(0, 1), (2, 2), (1, 3), (2, 3)]
[(0, 2), (9, 2), (8, 1), (1, 3), (8, 3), (2, 1), (7, 3), (3, 3), (6, 1), (4, 2), (6, 1), (5, 4), (6, 1), (6, 1)]


In [49]:
def weighted_id_sum(sections):
    total = 0
    current_start = 0
    for sec_id, sec_length in sections:
        # sum of i from current_start to current_start+sec_length-1
        index_sum = sec_length * current_start + (sec_length * (sec_length - 1)) // 2
        total += sec_id * index_sum
        current_start += sec_length
    return total

weighted_id_sum(fill_empty_sections(input))

6337367222422